# The Experiment Container

This notebook explains how the database works as an experiment container.

In [1]:
import os

import qcodes as qc
from qcodes.dataset.sqlite.database import initialise_or_create_database_at

In [2]:
db_file_path = os.path.join(os.getcwd(), 'exp_container_tutorial.db')
initialise_or_create_database_at(db_file_path)

Upgrading database; v0 -> v1: : 0it [00:00, ?it/s]
Upgrading database; v1 -> v2: 100%|█████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.17it/s]
Upgrading database; v2 -> v3: : 0it [00:00, ?it/s]
Upgrading database; v3 -> v4: : 0it [00:00, ?it/s]
Upgrading database; v4 -> v5: 100%|█████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 199.91it/s]
Upgrading database; v5 -> v6: : 0it [00:00, ?it/s]
Upgrading database; v8 -> v9: 100%|█████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 142.75it/s]


## The experiments inside the database

The database holds a certain number of **experiments**. They may be viewed:

In [3]:
from qcodes.dataset.experiment_container import experiments
from qcodes.dataset.experiment_setting import get_active_experiment_id

In [4]:
experiments()

[]

Not surprisingly, our new database is empty. If no experiment is created or loaded in the python kernel, the return value of `get_active_experiment_id()` will be None:

In [5]:
get_active_experiment_id()

Let us add some experiments:

In [6]:
from qcodes.dataset.experiment_container import new_experiment, load_or_create_experiment

In [7]:
new_experiment('first_exp', sample_name='old_sample')
new_experiment('second_exp', sample_name='slightly_newer_sample')
# A more convenient function that can load an experiment
# OR create a new one if it does not exist:
load_or_create_experiment('third_exp', sample_name='brand_new_sample')

third_exp#brand_new_sample#3@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
-----------------------------------------------------------------------------------

In [8]:
experiments()

[first_exp#old_sample#1@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
 -----------------------------------------------------------------------------,
 second_exp#slightly_newer_sample#2@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
 -----------------------------------------------------------------------------------------,
 third_exp#brand_new_sample#3@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
 -----------------------------------------------------------------------------------]

We notice that each experiment is labelled by an integer. This is the **exp_id** that can be used when looking up properties of each experiment.

Let's check again which experiment is active:

In [9]:
get_active_experiment_id()

3

The latest created or loaded experiment in the kernel becomes the active experiment and the function returns the `exp_id` of the active experiment.

Let us add some runs to our experiments. For the sake of clarity, we don't add any data to the runs here. We refer to the [Performing measurements using qcodes parameters and dataset](Performing-measurements-using-qcodes-parameters-and-dataset.ipynb) notebook for detailed information on how to properly create and populate runs. We also refer to the [DataSet-class-walkthrough](DataSet-class-walkthrough.ipynb) notebook for more details on the ``DataSet`` class. Note that the ``new_data_set`` function is used here ONLY for the sake of exercise and should NOT be used in the actual experiment.

In [10]:
from qcodes.dataset.data_set import new_data_set

In [11]:
new_data_set('run_a')

run_a #1@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
---------------------------------------------------------------

In a QCoDeS run, the run will be assigned to the active experiment in the kernel, if `exp_id` is not supplied explicitly in the run as a keyword argument. Let's load the active experiment and check the content

In [12]:
from qcodes.dataset.experiment_container import load_experiment

In [13]:
exp = load_experiment(3)

In [14]:
# Printing the experiment will reveal the runs
print(exp)

third_exp#brand_new_sample#3@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
-----------------------------------------------------------------------------------
1-run_a-1--0


We load one of experiments created above, and add a few runs to it:

In [15]:
exp2 = load_or_create_experiment('second_exp', sample_name='slightly_newer_sample')

This is the the second created experiment in the database, so we expect the active `exp_id` to be 2:

In [16]:
get_active_experiment_id()

2

In [17]:
new_data_set('run_b')

run_b #2@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
---------------------------------------------------------------

In [18]:
new_data_set('run_c')

run_c #3@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
---------------------------------------------------------------

Let's check the content of this experiment:

In [19]:
print(exp2)

second_exp#slightly_newer_sample#2@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
-----------------------------------------------------------------------------------------
2-run_b-1--0
3-run_c-2--0


We can also explicitly use `exp_id` in creating runs:

In [20]:
new_data_set('first_run', exp_id=1)

first_run #4@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
-------------------------------------------------------------------

Now, the alert reader will have noticed that `exp_id` is a keyword argument in QCoDeS runs. What is the default then? -- By default, the maximum `exp_id` in the database is the default value, and the experiment with that `exp_id` will be used in QCoDeS runs. However, if the user loads or creates an experiment in the Python session, that experiment will be activated and used in runs.

To show how default value works, we use a helper function to disable the active experiment in the kernel:

In [21]:
from qcodes.dataset.experiment_setting import reset_active_experiment_id

In [22]:
reset_active_experiment_id()

Now, the active experiment should show nothing, which means that the mentioned default `exp_id` will be used in runs:

In [23]:
get_active_experiment_id()

Now, we create a new_data_set without passing `exp_id`, we expect `exp_id=3` to be used in the run as it is the last experiment in our database:

In [24]:
new_data_set('default_run')

default_run #5@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
---------------------------------------------------------------------

Let's load the `exp_id=3` and print its content:

In [25]:
exp = load_experiment(3)

In [26]:
print(exp)

third_exp#brand_new_sample#3@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
-----------------------------------------------------------------------------------
1-run_a-1--0
5-default_run-2--0
